# Pars 1: URL to transcript, yt hash

In [1]:
from bs4 import BeautifulSoup
import numpy as np

import requests

url = "https://www.juznevesti.com/15-minuta/Marko-Nedeljkovic.sr.html"

req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")

In [2]:
text = ""
for i in soup.find_all("div"):
    current_classes_set = set(i.get("class", list()))
    if current_classes_set == {"bbredbolder", "mb20", "desc_holder"}:
        for j in i.find_all("p"):
            text += j.text + "\n"
video_div = soup.find(id="video_1")
video_link = video_div.contents[1].get("src")
yt_hash = video_link.split("/")[-1]
yt_hash


'Sk4JedgwClY'

In [3]:
video_div.contents

['\n',
 <iframe allowfullscreen="" frameborder="0" height="265" src="https://www.youtube.com/embed/Sk4JedgwClY" width="470"></iframe>,
 '\n']

# Pars 2: generating URLs.

In [31]:
found_urls = []
for pagenumber in range(1, 16):
    
    url = f"https://www.juznevesti.com/15-minuta/index.{pagenumber}.sr.html"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    divs = soup.find_all("div")
    for div in divs:
        found_urls.extend([i.get("href") for i in div.find_all("a")])
found_urls = list(set(found_urls))


# Fusion of the two parts

In [32]:
transcripts = list()
hashes = list()

for url in found_urls:
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    current_text = ""
    for i in soup.find_all("div"):
        current_classes_set = set(i.get("class", list()))
        if "desc_holder" in current_classes_set:
            current_text = "    ".join([j.text + "\n" for j in i.find_all("p")])
    # video_div = soup.find(id="video_1")
    # for element in video_div.contents:
    #     try:
    #         video_link = element.get("src", np.nan)
    #         break
    #     except:
    #         continue
    iframes = soup.find_all("iframe")

    video_link = [i.get("src") for i in iframes  if "youtube.com" in i.get("src")][0]
    yt_hash = video_link.split("/")[-1]
    transcripts.append(current_text)
    hashes.append(yt_hash)

IndexError: list index out of range

In [33]:
list(map(len, [hashes, transcripts, found_urls]))

[0, 0, 404]

In [ ]:

import pandas as pd
df = pd.DataFrame(data={

    "url": found_urls,
    "yt_hash": hashes,
    "transcript": transcripts,
})

df

In [ ]:
df.isna().sum(axis=0)

In [ ]:
df.to_csv("006_crawling_juznevesti.csv", index=False)